In [1]:
#Importting all the necessary libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime as datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
#loading the dataset
pizza = pd.read_excel("Data Model - Pizza Sales.xlsx")

In [3]:
pizza.head()

,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,2015-01-01,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [4]:
pizza.drop(columns=["order_details_id","order_id","pizza_name","pizza_ingredients","pizza_size","unit_price","pizza_category","pizza_id"],inplace=True)
pizza.head()

,quantity,order_date,order_time,total_price
0,1,2015-01-01,11:38:36,13.25
1,1,2015-01-01,11:57:40,16.00
2,1,2015-01-01,11:57:40,18.50
3,1,2015-01-01,11:57:40,20.75
4,1,2015-01-01,11:57:40,16.00


In [5]:
pizza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   quantity     48620 non-null  int64         
 1   order_date   48620 non-null  datetime64[ns]
 2   order_time   48620 non-null  object        
 3   total_price  48620 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.5+ MB


In [6]:
pizza["order_time"] = pd.to_datetime(pizza["order_time"], format="%H:%M:%S")
pizza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   quantity     48620 non-null  int64         
 1   order_date   48620 non-null  datetime64[ns]
 2   order_time   48620 non-null  datetime64[ns]
 3   total_price  48620 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 1.5 MB


In [7]:
pizza["Day_name"] = pizza["order_date"].dt.day_name()

pizza["Weekend"] = np.where(pizza["Day_name"].isin(["Saturday","Sunday"]),1,0) #Making 1 is it is weekend else 0
pizza.sample(3)

,quantity,order_date,order_time,total_price,Day_name,Weekend
2713,1,2015-01-21,1900-01-01 12:23:38,16.0,Wednesday,0
48532,1,2015-12-31,1900-01-01 17:16:32,12.0,Thursday,0
32861,1,2015-08-31,1900-01-01 12:47:52,16.0,Monday,0


In [8]:
pizza["Date"] = pizza["order_date"].dt.day
pizza["Month_name"] = pizza["order_date"].dt.month_name()
pizza.sample(3)

,quantity,order_date,order_time,total_price,Day_name,Weekend,Date,Month_name
39808,1,2015-10-25,1900-01-01 12:39:32,20.50,Sunday,1,25,October
21111,1,2015-06-04,1900-01-01 20:59:56,16.00,Thursday,0,4,June
40357,1,2015-10-30,1900-01-01 18:20:42,20.75,Friday,0,30,October


In [9]:
morning_start = pd.to_datetime('04:00:00').time()
afternoon_start = pd.to_datetime('12:00:00').time()
night_start = pd.to_datetime('18:00:00').time()

# Defining a function to determine the time phase
def get_time_phase(time):
    if time >= morning_start and time < afternoon_start:
        return 'Morning'
    elif time >= afternoon_start and time < night_start:
        return 'Afternoon'
    else:
        return 'Night'

# Apply the function to create the new column
pizza['time_phase'] = pizza['order_time'].dt.time.apply(get_time_phase)

# Display the updated DataFrame with the new column
print(pizza[['order_time', 'time_phase']].sample(10))
pizza

               order_time time_phase
21875 1900-01-01 17:15:36  Afternoon
9662  1900-01-01 12:26:06  Afternoon
40238 1900-01-01 20:45:04      Night
7318  1900-01-01 11:56:22    Morning
15333 1900-01-01 17:14:03  Afternoon
8172  1900-01-01 12:42:52  Afternoon
15053 1900-01-01 14:29:19  Afternoon
48112 1900-01-01 11:41:28    Morning
20612 1900-01-01 14:31:16  Afternoon
333   1900-01-01 13:41:36  Afternoon


,quantity,order_date,order_time,total_price,Day_name,Weekend,Date,Month_name,time_phase
0,1,2015-01-01,1900-01-01 11:38:36,13.25,Thursday,0,1,January,Morning
1,1,2015-01-01,1900-01-01 11:57:40,16.00,Thursday,0,1,January,Morning
2,1,2015-01-01,1900-01-01 11:57:40,18.50,Thursday,0,1,January,Morning
3,1,2015-01-01,1900-01-01 11:57:40,20.75,Thursday,0,1,January,Morning
4,1,2015-01-01,1900-01-01 11:57:40,16.00,Thursday,0,1,January,Morning
...,...,...,...,...,...,...,...,...,...
48615,1,2015-12-31,1900-01-01 21:23:10,16.75,Thursday,0,31,December,Night
48616,1,2015-12-31,1900-01-01 21:23:10,17.95,Thursday,0,31,December,Night
48617,1,2015-12-31,1900-01-01 21:23:10,12.00,Thursday,0,31,December,Night
48618,1,2015-12-31,1900-01-01 22:09:54,20.25,Thursday,0,31,December,Night


In [10]:
morning_data = pizza[pizza['time_phase']=='Morning']
afternoon_data = pizza[pizza['time_phase']=='Afternoon']
night_data = pizza[pizza['time_phase']=='Night']

morning_sales = morning_data.groupby(["order_date","time_phase"])[["quantity","total_price"]].sum().reset_index()
afternoon_sales = afternoon_data.groupby(["order_date","time_phase"])[["quantity","total_price"]].sum().reset_index()
night_sales = night_data.groupby(["order_date","time_phase"])[["quantity","total_price"]].sum().reset_index()

# pz_1 = morning_sales.merge(afternoon_sales,on="order_date",how="outer")
# pz_1 = pz_1.merge(night_sales,on="order_date",how="outer")
pz_1 = pd.concat([morning_sales,afternoon_sales,night_sales])
pz_1.sort_values(by="order_date",inplace=True)
pz_1.fillna(0,inplace=True)
# pz_1.to_excel("finally_done.xlsx")
pz_1.head()

,order_date,time_phase,quantity,total_price
0,2015-01-01,Morning,6,105.25
0,2015-01-01,Afternoon,106,1757.35
0,2015-01-01,Night,50,851.25
1,2015-01-02,Night,82,1358.95
1,2015-01-02,Morning,5,87.25


In [11]:
pz_1["Date"] = pz_1["order_date"].dt.day
pz_1["Month_name"] = pz_1["order_date"].dt.month_name()
pz_1["Day_name"] = pz_1["order_date"].dt.day_name()
pz_1["Weekend"] = np.where(pz_1["Day_name"].isin(["Saturday","Sunday"]),1,0) #Making 1 is it is weekend else 0
pz_1.sample(3)

,order_date,time_phase,quantity,total_price,Date,Month_name,Day_name,Weekend
156,2015-06-18,Morning,10,151.00,18,June,Thursday,0
249,2015-09-28,Morning,8,132.25,28,September,Monday,0
331,2015-12-31,Morning,5,77.75,31,December,Thursday,0


In [12]:
pz_1.head()

,order_date,time_phase,quantity,total_price,Date,Month_name,Day_name,Weekend
0,2015-01-01,Morning,6,105.25,1,January,Thursday,0
0,2015-01-01,Afternoon,106,1757.35,1,January,Thursday,0
0,2015-01-01,Night,50,851.25,1,January,Thursday,0
1,2015-01-02,Night,82,1358.95,2,January,Friday,0
1,2015-01-02,Morning,5,87.25,2,January,Friday,0


In [13]:
pz_1.to_excel("main_pizza_dataset_ansh.xlsx",index=False)